In [1]:
from causalis.data import generate_rct_data
from causalis.data import CausalData
from causalis.inference import ttest

In [2]:
df = generate_rct_data()

In [3]:
df.columns

Index(['user_id', 'treatment', 'target', 'age', 'cnt_trans',
       'platform_Android', 'platform_iOS', 'invited_friend'],
      dtype='object')

In [4]:
cdata = CausalData(
    df=df,
    outcome='outcome',
    confounders=['age', 'cnt_trans',
       'platform_Android', 'platform_iOS', 'invited_friend'],
    treatment='treatment'
)

In [9]:
cdata.metadata

{}

In [5]:
ttest(cdata)

{'p_value': np.float64(0.00017501995114700586),
 'absolute_difference': np.float64(0.0165),
 'absolute_ci': (np.float64(0.007883204030438051),
  np.float64(0.02511679596956195)),
 'relative_difference': np.float64(16.467065868263475),
 'relative_ci': (np.float64(7.867469092253545),
  np.float64(25.066662644273407))}

In [6]:
df.groupby('treatment')['outcome'].mean()

treatment
0    0.1002
1    0.1167
Name: target, dtype: float64

In [7]:
"""
Test script to verify the ttest function in the inference module.
"""

import pandas as pd
import numpy as np
from causalis.data import CausalData
from causalis.inference.ttest import ttest

# Set random seed for reproducibility
np.random.seed(42)

# Create a test DataFrame with a known effect size
n = 1000
control_mean = 10.0
treatment_effect = 2.0
treatment_mean = control_mean + treatment_effect

# Create data with treatment effect
df = pd.DataFrame({
    'user_id': range(1, n + 1),
    'treatment': np.random.choice([0, 1], size=n),
    'age': np.random.randint(18, 65, size=n),
    'gender': np.random.choice(['M', 'F'], size=n),
})

# Generate outcome variable with treatment effect
df['outcome'] = np.where(
    df['treatment'] == 1,
    np.random.normal(treatment_mean, 2.0, size=n),  # Treatment group
    np.random.normal(control_mean, 2.0, size=n)     # Control group
)

# Create a causaldata object
ck = CausalData(
    df=df,
    outcome='outcome',
    confounders=['age', 'gender'],
    treatment='treatment'
)

# Test 1: Basic functionality
print("Test 1: Basic functionality")
result = ttest(ck)
print(f"p_value: {result['p_value']}")
print(f"absolute_difference: {result['absolute_difference']}")
print(f"absolute_ci: {result['absolute_ci']}")
print(f"relative_difference: {result['relative_difference']}")
print(f"relative_ci: {result['relative_ci']}")
print()

# Test 2: Check if absolute difference is close to the expected treatment effect
print("Test 2: Check if absolute difference is close to the expected treatment effect")
expected_diff = treatment_effect
actual_diff = result['absolute_difference']
diff_error = abs(actual_diff - expected_diff)
print(f"Expected difference: {expected_diff}")
print(f"Actual difference: {actual_diff}")
print(f"Error: {diff_error}")
print(f"Test passed: {diff_error < 0.5}")  # Allow for some random variation
print()

# Test 3: Check if confidence intervals contain the true effect
print("Test 3: Check if confidence intervals contain the true effect")
lower_bound, upper_bound = result['absolute_ci']
contains_true_effect = lower_bound <= expected_diff <= upper_bound
print(f"Confidence interval: ({lower_bound:.4f}, {upper_bound:.4f})")
print(f"True effect: {expected_diff}")
print(f"CI contains true effect: {contains_true_effect}")
print()

# Test 4: Check relative difference
print("Test 4: Check relative difference")
expected_rel_diff = (treatment_effect / control_mean) * 100
actual_rel_diff = result['relative_difference']
rel_diff_error = abs(actual_rel_diff - expected_rel_diff)
print(f"Expected relative difference: {expected_rel_diff:.2f}%")
print(f"Actual relative difference: {actual_rel_diff:.2f}%")
print(f"Error: {rel_diff_error:.2f}%")
print(f"Test passed: {rel_diff_error < 5}")  # Allow for some random variation
print()

# Test 5: Different confidence level
print("Test 5: Different confidence level")
result_90 = ttest(ck, confidence_level=0.90)
result_99 = ttest(ck, confidence_level=0.99)

# 90% CI should be narrower than 95% CI (default)
ci_width_90 = result_90['absolute_ci'][1] - result_90['absolute_ci'][0]
ci_width_95 = result['absolute_ci'][1] - result['absolute_ci'][0]
ci_width_99 = result_99['absolute_ci'][1] - result_99['absolute_ci'][0]

print(f"90% CI width: {ci_width_90:.4f}")
print(f"95% CI width: {ci_width_95:.4f}")
print(f"99% CI width: {ci_width_99:.4f}")
print(f"Test passed: {ci_width_90 < ci_width_95 < ci_width_99}")
print()

# Test 6: Error handling - no treatment
print("Test 6: Error handling - no treatment")
ck_no_treatment = CausalData(
    df=df,
    outcome='outcome',
    confounders=['age', 'gender']
)
try:
    result = ttest(ck_no_treatment)
    print("Error: Test failed - should have raised ValueError")
except ValueError as e:
    print(f"Success: Correctly raised ValueError: {e}")
print()

# Test 7: Error handling - no outcome
print("Test 7: Error handling - no outcome")
ck_no_target = CausalData(
    df=df,
    confounders=['age', 'gender'],
    treatment='treatment'
)
try:
    result = ttest(ck_no_target)
    print("Error: Test failed - should have raised ValueError")
except ValueError as e:
    print(f"Success: Correctly raised ValueError: {e}")
print()

# Test 8: Error handling - non-binary treatment
print("Test 8: Error handling - non-binary treatment")
df_multi = df.copy()
df_multi['treatment'] = np.random.choice([0, 1, 2], size=n)
ck_multi = CausalData(
    df=df_multi,
    outcome='outcome',
    confounders=['age', 'gender'],
    treatment='treatment'
)
try:
    result = ttest(ck_multi)
    print("Error: Test failed - should have raised ValueError")
except ValueError as e:
    print(f"Success: Correctly raised ValueError: {e}")
print()

print("All tests completed.")

Test 1: Basic functionality
p_value: 1.4940213443180423e-50
absolute_difference: 1.9617748200899392
absolute_ci: (np.float64(1.718663386392377), np.float64(2.2048862537875014))
relative_difference: 19.648731193051145
relative_ci: (np.float64(17.213777312635003), np.float64(22.083685073467286))

Test 2: Check if absolute difference is close to the expected treatment effect
Expected difference: 2.0
Actual difference: 1.9617748200899392
Error: 0.0382251799100608
Test passed: True

Test 3: Check if confidence intervals contain the true effect
Confidence interval: (1.7187, 2.2049)
True effect: 2.0
CI contains true effect: True

Test 4: Check relative difference
Expected relative difference: 20.00%
Actual relative difference: 19.65%
Error: 0.35%
Test passed: True

Test 5: Different confidence level
90% CI width: 0.4079
95% CI width: 0.4862
99% CI width: 0.6395
Test passed: True

Test 6: Error handling - no treatment
Success: Correctly raised ValueError: causaldata object must have a treatmen